In [2]:
import pandas as pd
import numpy as np
import nbimporter
from datetime import datetime, timedelta
import glob
import os
import warnings
from sklearn.model_selection import TimeSeriesSplit
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras.optimizers import Adam

warnings.simplefilter(action='ignore', category=FutureWarning)

# 1. LSTM

## 1.1 Sequenzen erstellen

In [1]:
def create_sequences(data, target, sequence_length):
    """
    Erstellt Sequenzen aus den gegebenen Daten für die Verwendung in zeitlichen Vorhersagemodellen.

    Parameter:
    data (array-ähnlich): Die Eingabedaten, aus denen die Sequenzen erstellt werden.
    target (array-ähnlich): Die Zielwerte, die den Sequenzen folgen und vorhergesagt werden sollen.
    sequence_length (int): Die Länge jeder Sequenz, also die Anzahl der Schritte in der Sequenz.

    Rückgabewert:
    tuple: Ein Tupel bestehend aus zwei Arrays:
        - sequences (array): Ein Array von Sequenzen mit der Länge 'sequence_length'.
        - labels (array): Ein Array der Zielwerte, die jeweils auf eine Sequenz folgen.
    """
    
    sequences = []
    labels = []
    
    # Iteriere über die Daten und erstelle Sequenzen mit der gegebenen Länge
    for i in range(len(data) - sequence_length):
        seq = data[i:i + sequence_length]  # Eingabesequenz
        label = target[i + sequence_length]  # Zielwert, der auf die Sequenz folgt
        sequences.append(seq)
        labels.append(label)
    
    # Rückgabe der Sequenzen und der entsprechenden Zielwerte als NumPy-Arrays
    return np.array(sequences), np.array(labels)


## 1.2 Modell aufsetzen

In [2]:
def lstm_model(X_train, X_test, y_train, y_test, layers, epochs, regularization, dropout, learning_rate, early_stopping):
    """
    Erstellt und trainiert ein LSTM-Modell zur Vorhersage von Zeitreihendaten.

    Parameter:
    X_train (array-ähnlich): Trainingsdaten für die Eingabevariablen.
    X_test (array-ähnlich): Testdaten für die Eingabevariablen.
    y_train (array-ähnlich): Trainingsdaten für die Zielvariable.
    y_test (array-ähnlich): Testdaten für die Zielvariable.
    layers (int): Anzahl der LSTM-Einheiten in den LSTM-Schichten.
    epochs (int): Anzahl der Epochen, die das Modell trainiert wird.
    regularization (float): L2-Regularisierungswert zur Vermeidung von Überanpassung.
    dropout (float): Dropout-Rate, die während des Trainings verwendet wird.
    learning_rate (float): Lernrate für den Adam-Optimierer.
    early_stopping (int): Geduld für Early Stopping, die bestimmt, wie viele Epochen ohne Verbesserung der Validierungsleistung toleriert werden.

    Rückgabewert:
    tuple: Ein Tupel bestehend aus:
        - lstm_model: Das Ergebnis des Trainingsprozesses (History-Objekt).
        - model: Das trainierte LSTM-Modell.
    """

    # Modellinitialisierung
    model = Sequential()

    # Erste LSTM-Schicht mit Rückgabe von Sequenzen
    model.add(LSTM(layers, return_sequences=True, input_shape=(X_train.shape[1], X_train.shape[2]),
                   kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout))

    # Zweite LSTM-Schicht ohne Rückgabe von Sequenzen
    model.add(LSTM(layers, return_sequences=False, kernel_regularizer=l2(regularization)))
    model.add(Dropout(dropout))

    # Dense-Schicht mit 25 Neuronen und ReLU-Aktivierung
    model.add(Dense(25, activation='relu', kernel_regularizer=l2(regularization)))

    # Letzte Dense-Schicht mit einer Einheit (für die Vorhersage)
    model.add(Dense(1))

    # Definition des Adam-Optimierers mit angegebener Lernrate
    optimizer = Adam(learning_rate=learning_rate)

    # Kompilieren des Modells mit dem Optimierer und der Verlustfunktion 'mean_squared_error'
    model.compile(optimizer=optimizer, loss='mean_squared_error')

    # Early Stopping zum Beenden des Trainings, wenn sich die Validierungsleistung nicht verbessert
    early_stopping = EarlyStopping(monitor='val_loss', patience=early_stopping, restore_best_weights=True)

    # Training des Modells mit den Trainingsdaten und Anwendung von Early Stopping
    lstm_model = model.fit(X_train, y_train, epochs=epochs, batch_size=20, validation_data=(X_test, y_test),
                           callbacks=[early_stopping], verbose=1)

    # Rückgabe des Trainingsverlaufs und des trainierten Modells
    return lstm_model, model
